## Consulta 03

Esta consulta devuelve devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Endpoint: def UserForGenre( genero : str ):\
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



*Importamos la librería y módulo necesarios para el procesamiento de los datos*

In [1]:
import pandas as pd
import csv

*Importamos los archivos con los que vamos a trabajar la consulta*

In [2]:
df_games = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_games_clean.parquet')
df_items = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_items_clean.parquet')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [3]:
df_games_copy = df_games.copy()
df_items_copy = df_items.copy()

*(Opcional). Podemos hacer una visualización previa del dataset para hacer cualquier análsis previo.*

In [4]:
df_games_copy

,id,app_name,genres,price,release_date,developer
88310,761140,Lost Summoner Kitty,"Action, Casual, Indie, Simulation, Strategy",4.99,2018-01-04,Kotoshiro
88311,643980,Ironbound,"Indie, RPG, Strategy",NaN,2018-01-04,Secret Level SRL
88312,670290,Real Pool 3D - Poolians,"Casual, Indie, Simulation, Sports",NaN,2017-07-24,Poolians.com
88313,767400,弹炸人2222,"Action, Adventure, Casual",0.99,2017-12-07,彼岸领域
88314,773570,Log Challenge,"Action, Indie, Casual, Sports",2.99,None,None
...,...,...,...,...,...,...
120440,773640,Colony On Mars,"Casual, Indie, Simulation, Strategy",1.99,2018-01-04,"Nikita ""Ghost_RUS"""
120441,733530,LOGistICAL: South Africa,"Casual, Indie, Strategy",4.99,2018-01-04,Sacada
120442,610660,Russian Roads,"Indie, Racing, Simulation",1.99,2018-01-04,Laush Dmitriy Sergeevich
120443,658870,EXIT 2 - Directions,"Casual, Indie",4.99,2017-09-02,"xropi,stev3ns"


In [5]:
df_items_copy

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


*Desarrollamos la función correspondiente para hacer la consulta que necesitamos*

In [6]:
def UserForGenre(genero: str, df_games_copy, df_items_copy):
    # Verificar si la columna 'genre' está presente en el DataFrame df_games_copy
    if 'genres' not in df_games_copy.columns:
        raise ValueError("El DataFrame df_games_copy no tiene una columna llamada 'genre'.")

#Convertir la columna 'release_date' a tipo datetime
    df_games_copy['release_date'] = pd.to_datetime(df_games_copy['release_date'], errors='coerce')

#Filtrar df_games_copy por el género dado
    juegos_genero = df_games_copy[df_games_copy['genres'] == genero]

#Unir el DataFrame filtrado con df_user_items
    juegos_usuario = juegos_genero.merge(df_items_copy, left_on='id', right_on='item_id')

#Calcular las horas jugadas por usuario para cada juego
    horas_por_usuario = juegos_usuario.groupby('user_id')['playtime_forever'].sum().reset_index()

#Encontrar el usuario con más horas jugadas
    usuario_max_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax()]['user_id']

#Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado
    horas_por_año = juegos_usuario.groupby(juegos_usuario['release_date'].dt.year)['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'playtime_forever': 'Horas'}, inplace=True)
    horas_por_año = horas_por_año.to_dict('records')

#Crear el diccionario de resultados
    result = {
        "Usuario con más horas jugadas para {}: ".format(genero): usuario_max_horas,
        "Horas jugadas": horas_por_año
    }

    return result

*Aplicamos la función y corroboramos que funcione correctamente*

In [7]:
#Ejemplo de uso:
resultado = UserForGenre("Action", df_games_copy, df_items_copy)
print(resultado)

{'Usuario con más horas jugadas para Action: ': 'Sp3ctre', 'Horas jugadas': [{'release_date': 1990.0, 'Horas': 18718.0}, {'release_date': 1991.0, 'Horas': 946.0}, {'release_date': 1992.0, 'Horas': 120.0}, {'release_date': 1993.0, 'Horas': 4705.0}, {'release_date': 1994.0, 'Horas': 114347.0}, {'release_date': 1995.0, 'Horas': 186479.0}, {'release_date': 1996.0, 'Horas': 20189.0}, {'release_date': 1997.0, 'Horas': 284555.0}, {'release_date': 1998.0, 'Horas': 2797035.0}, {'release_date': 1999.0, 'Horas': 1820638.0}, {'release_date': 2000.0, 'Horas': 18507806.0}, {'release_date': 2001.0, 'Horas': 1145463.0}, {'release_date': 2002.0, 'Horas': 576432.0}, {'release_date': 2003.0, 'Horas': 3889960.0}, {'release_date': 2004.0, 'Horas': 131145171.0}, {'release_date': 2005.0, 'Horas': 11952680.0}, {'release_date': 2006.0, 'Horas': 4230509.0}, {'release_date': 2007.0, 'Horas': 12438095.0}, {'release_date': 2008.0, 'Horas': 23669497.0}, {'release_date': 2009.0, 'Horas': 144433841.0}, {'release_date